# CS342 Machine Learning
# Lab 4: Linear regression

## Department of Computer Science, University of Warwick

This lab focuses on the use of regularization for linear regression.

# Data files for the lab

If working on one of the DCS machines, the data may be found here:

```/modules/cs342/2020/lab4/data/prostate_data.csv ```

You may load the data directly from that directory.

If you are using your own machine, copy the data across by running the following command in a terminal window using the remote node corresponding to your username. The name of this remote node uses the last two digits of your username in the form remote-nn, for example, if your username is u1234567 you would connect to remote-67.dcs.warwick.ac.uk (recall to use your USERNAME and correpsonding REMOTE_NN):

```scp USERNAME@REMOTE_NN.dcs.warwick.ac.uk:/modules/cs342/2020/lab4/data/prostate_data.csv .```

After entering your DCS password, this will copy the data to your current working directory. You should now have the following file:
```
├──[your working directory]
   └── prostate_data.csv
```
**Please make sure to use the correct path to these files when working on your own machine. The scripts below assume you are working on the DCS machines. Recall that the *.ipynb file (this file) should be in your working directory.**

The prostate dataset (file *prostate_data.csv* see: https://web.stanford.edu/~hastie/ElemStatLearn//datasets/prostate.data) will be used to predict the numerical target variable *lpsa* based on 8 features (*lcavol, lweight, age, lbph, svi, lcp, gleason, pgg45*). There are 97 observations in total. The last column is a Train/Predict flag to be used to separate the observations into two subsets. The *train = T* subset will be used for model fitting and cross-validation (CV), while the *train = F* subset will be used for testing after model selection and training.

Import the data into a Pandas data frame and standardize the features. 

In [1]:
#import prostate dataset



#standardise features


### Non-regularized linear regression 

Scikit-learn has a plethora of linear models:http://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model. It includes all the implementations needed for this lab, including implementations that employ cross-validation to select hyper-parameters for regularization. 

Fit a non-regularized linear regression model to the *train = T* subset. Use your fitted model to predict the target variable in the *train = F* subset. 

In [2]:
#Fit and test a non-regularized linear regression model


### L2-regularized linear regression (ridge regression)

Fit an L2-regularized linear regression model to the *train = T* subset once you use 3-fold CV  to tune the hyper-parameter for regularization. Use the model fitted with the best hyper-parameter to predict the target variable in the *train = F* subset. 

In [2]:
#Fit and test an L2-regularized linear regression model


### L1-regularized linear regression (lasso regression)

Fit an L1-regularized linear regression model to the *train = T* subset once you use 3-fold CV  to tune the hyper-parameter for regularization. Use the model fitted with the best hyper-parameter to predict the target variable in the *train = F* subset. 

In [3]:
#Fit and test an L1-regularized linear regression model


1. Which model performes the best for this dataset?
2. Which features are irrelevant for this task? **Hint:** display the learned coefficients (weights) for each model and recall which type of regularization allows for feature selection.